# Suivi de trajectoire - Démonstration (Resnet18)

Dans ce notebook, nous allons utiliser le modèle entrainé en situation réelle !

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras

# Création du modèle

In [ ]:
import classification_models
from classification_models.tfkeras import Classifiers

# Chargement du modèle ResNEt18
ResNet18, preprocess_input = Classifiers.get('resnet18')

# Instanciation du modèle pré-entrainé ResNet18
base_model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=False,pooling=False)
  
# Ajout de l'applatissement des sorties et de la couche dense avec 2 neurones"
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(units=2, activation='linear')(x)

model = tf.keras.Model(inputs=[base_model.input], outputs=[output])

On charge ensuite les poids sauvegardés lors de l'entrainement :

In [ ]:
model.load_weights("meilleur_modele.hdf5")

# Création de la classe Camera

J'ai créé une classe Image personnelle afin d'optimiser le pipeline avec OpenCV :

In [ ]:
import traitlets
import threading
import atexit
import numpy as np


class Camera(traitlets.HasTraits):
    type_camera = traitlets.Unicode("CSI")
    capture_device = traitlets.Integer(default_value=0)
    capture_width = traitlets.Integer(default_value=1280)
    capture_height = traitlets.Integer(default_value=720)
    display_width = traitlets.Integer(default_value=640)
    display_height = traitlets.Integer(default_value=480)
    fps = traitlets.Integer(default_value=30)
    flip = traitlets.Integer(default_value=0)
    image = traitlets.Any()
    video_on = traitlets.Bool(default_value=False)
    
    def __init__(self,*args,**kwargs):
        super(Camera, self).__init__(*args, **kwargs)
        self._running = False
        self.image = np.zeros((self.display_height, self.display_width, 3), dtype=np.uint8)
        
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)

        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)
    
    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            image_resized = cv2.resize(image,(int(self.display_width),int(self.display_height)))
            return image_resized
        else:
            return self.image
    
    # ON/OFF de la capture vidéo
    def capture_video(self,run=False):
        if run is True:
            self.video_on = True
        else:
            self.video_on = False
    
    # Lecture d'un flux vidéo
    def _capture_video(self):
        while True:
            if not self._running:
                break
            self.image = self.capture_image()

            
    # Détachement de la caméra
    def release(self):
        self.cap.release()

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))
    
    # Surveillance de la variable "video_on"
    @traitlets.observe('video_on')
    def _on_running(self, change):
        if change['new'] and not change['old']:
            # not running -> running
            self._running = True
            self.thread = threading.Thread(target=self._capture_video)
            self.thread.start()
        elif change['old'] and not change['new']:
            # running -> not running
            self._running = False
            self.thread.join()

Fonction pour initialiser la caméra :

In [ ]:
def InitCamera():
    camera = Camera(type_camera="CSI",capture_device=0,
                capture_width=224,capture_height=224,
                display_width=224,display_height=224,
                fps=25,flip=0)
    return camera

# Essai vidéo de prédictions

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display
from jetbot import bgr8_to_jpeg

print("Initialisation de la caméra...")
# Initialise la caméra
try :
    camera.capture_video(run=False)
    camera.release()
    del camera
except NameError:
    pass
camera = InitCamera()

print("Création des widget...")
# Création du widget de la vidéo
camera_widget = ipywidgets.Image(format="jpeg",width=camera.display_width, height=camera.display_height, value=bgr8_to_jpeg(camera.image))
traitlets.dlink((camera, 'image'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
camera_link = traitlets.dlink((camera, 'image'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# Création du widget de visualisaiton des coordonnées estimées
preview_widget = ipywidgets.Image(format="jpeg",width=camera.display_width, height=camera.display_height, value=bgr8_to_jpeg(camera.image))

# Création du widget "state"
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')

# Lance la caméra
camera.capture_video(run=True)

# Fonction de traitement des actions du widget "state_widget"
def live(state_widget,preview_widget, model, camera, essai = False):
    if essai == True:
        image = camera.image
        xy = model(tf.expand_dims(tf.cast(image,tf.uint8),0)).numpy()[0]
        
    while state_widget.value == 'live':
        # Capture de l'image
        image = camera.image
        
        # Prédiciton des coordonnées
        xy = model(tf.expand_dims(tf.cast(image,tf.uint8),0)).numpy()[0]

        # Calcul des coordonnées en pixel
        xs = np.int(camera.display_width * ((xy[0]+1) / 2.0))
        ys = np.int(camera.display_height * ((xy[1]+1) / 2.0))
        
        # Tracé de la cible sur l'image
        prediction = image.copy()
        prediction = cv2.circle(prediction, (xs, ys), 8, (255, 0, 0), 3)
        preview_widget.value = bgr8_to_jpeg(prediction)
       
def start_live(change):
    if change['new'] == 'live':
        global execute_thread
        execute_thread = threading.Thread(target=live, args=(state_widget, preview_widget, model, camera))
        execute_thread.start()
    else:
        execute_thread.join()

# Exécute une fois le modèle
print("Initialisation du modèle...")
execute_thread = threading.Thread(target=live, args=(state_widget, preview_widget, model, camera, True))
execute_thread.start()
execute_thread.join()
print("Modèle initialisé...")

Affiche l'interface :

In [ ]:
# Création du lien d'observation du widget state
state_widget.observe(start_live, names='value')

# Affichage de l'interface
data_collection_widget = ipywidgets.HBox([camera_widget,preview_widget])
display(state_widget,data_collection_widget)

# Ajout du pilotage automatique

### Algorithme de pilotage automatique

Les variables utilisées dans le calcul du pilotage automatique sont les suivantes :
- Gain_Vitesse : C'est le gain en vitesse apporté au robot
- Offset_Vitesse : C'est l'offset de vitesse à ajouter au calculateur
- Kp : C'est le gain propotionnel du PID
- Kd : C'est le gain dérivé du PID
- Ki : C'est le gain intégral du PID

In [ ]:
# Calcul de la commande automatique
# pos_xy : Coordonnées [-1.0,1.0] de la cible

nbr_step = 0
erreur_1 = 0.0

def CalculCommandes(pos_xy, Kp, Ki, Kd, Gain_Vitesse, Offset_Vitesse):
    global erreur_1
    global nbr_step
       
    # Calcul des coordonnées en pixel
    xs = np.int(camera.display_width * ((pos_xy[0]+1) / 2.0))
    ys = np.int(camera.display_height * ((pos_xy[1]+1) / 2.0))

    # Calcul de l'angle de la cible par rapport à la verticale
    angle = np.arctan2(camera.display_width/2 - xs,camera.display_height - ys)
    
    # Calcul de l'erreur
    erreur = 0 - angle

    # Calcul de la commande en sortie du PID
    commande = erreur * (Kp + Ki*((1/(nbr_step+1))*(erreur_1+erreur)) + Kd*(erreur - erreur_1))
    
    # Ajout de l'offset en vitesse
    commande = commande + Offset_Vitesse
    
    # Mise à jour des variables
    erreur_1 = erreur
    nbr_step = nbr_step + 1
    
    # Calcul des commandes moteur gauche et droit
    commande_gauche = max(min(Gain_Vitesse + commande, 1.0), 0.0)
    commande_droite = max(min(Gain_Vitesse - commande, 1.0), 0.0)
    
    return([commande_gauche,commande_droite])

### Interface de visualisation

On reprend l'interface précédent et on ajoute le calcul de la commande des moteurs ainsi que les sliders permettant de gérer les différents gains :

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
import ipywidgets

# Création des sliders
Gain_Vitesse_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='Gain vitesse')
Offset_Vitesse_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='Offset vitesse')
Commande_moteur_gauche = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='Mot. gauche',orientation='vertical')
Commande_moteur_droit = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='Mot. droit',orientation='vertical')

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display
from jetbot import bgr8_to_jpeg

# Gain du correcteur PID
Kp = 0.05
Ki = 0.0
Kd = 0.1

print("Initialisation de la caméra...")
# Initialise la caméra
try :
    camera.capture_video(run=False)
    camera.release()
    del camera
except NameError:
    pass
camera = InitCamera()

print("Création des widget...")
# Création du widget de la vidéo
camera_widget = ipywidgets.Image(format="jpeg",width=camera.display_width, height=camera.display_height, value=bgr8_to_jpeg(camera.image))
traitlets.dlink((camera, 'image'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
camera_link = traitlets.dlink((camera, 'image'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# Création du widget de visualisaiton des coordonnées estimées
preview_widget = ipywidgets.Image(format="jpeg",width=camera.display_width, height=camera.display_height, value=bgr8_to_jpeg(camera.image))

# Création du widget "state"
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')

# Lance la caméra
camera.capture_video(run=True)

# Fonction de traitement des actions du widget "state_widget"
def live(state_widget,preview_widget, model, camera, essai = False):
    global Kp
    global Ki
    global Kd
    global robot

    if essai == True:
        image = camera.image
        xy = model(tf.expand_dims(image,0)).numpy()[0]

    if state_widget.value is not 'live':
        robot.stop()
        
    while state_widget.value == 'live':
        # Capture de l'image
        image = camera.image
        
        # Prédiciton des coordonnées
        xy = model(tf.expand_dims(tf.cast(image,tf.uint8),0)).numpy()[0]
        
        # Calcul de la commande
        commande = CalculCommandes(xy, Kp, Ki, Kd, Gain_Vitesse_slider.value, Offset_Vitesse_slider.value)

        # Calcul des coordonnées en pixel
        xs = np.int(camera.display_width * ((xy[0]+1) / 2.0))
        ys = np.int(camera.display_height * ((xy[1]+1) / 2.0))
        
        # Tracé de la cible sur l'image
        prediction = image.copy()
        prediction = cv2.circle(prediction, (xs, ys), 8, (255, 0, 0), 3)
        preview_widget.value = bgr8_to_jpeg(prediction)
        
        # Commande du robot
        robot.left_motor.value = commande[0]
        robot.right_motor.value = commande[1]

def start_live(change):
    if change['new'] == 'live':
        global execute_thread
        execute_thread = threading.Thread(target=live, args=(state_widget, preview_widget, model, camera))
        execute_thread.start()
    else:
        execute_thread.join()

# Exécute une fois le modèle
print("Initialisation du modèle...")
execute_thread = threading.Thread(target=live, args=(state_widget, preview_widget, model, camera, True))
execute_thread.start()
execute_thread.join()
print("Modèle initialisé...")

In [ ]:
# Création du lien d'observation du widget state
state_widget.observe(start_live, names='value')

# Affichage de l'interface
commandes_widget = ipywidgets.HBox([Commande_moteur_gauche,Commande_moteur_droit])
data_collection_widget = ipywidgets.HBox([camera_widget,preview_widget,commandes_widget])

display(state_widget,data_collection_widget,Gain_Vitesse_slider,Offset_Vitesse_slider)

In [ ]:
robot.stop()